<a href="https://colab.research.google.com/github/AtlantisAnalytics/Grad-CAM/blob/main/gradCam_office31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grad-CAM visualisations for Office31 Dataset

## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/varun')

## Selecting particular folder and class from Office31 Dataset

In [ ]:
# importing the needed package
import os
from skimage import io
from warnings import filterwarnings
import tensorflow as tf
from tensorflow import io
from tensorflow import image
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
# Run the below cell to select a particular class

In [ ]:
try:
    print(os.listdir('office31'))
    print('Select any one out of these\n')
    x = input()
    print(os.listdir('office31/'+x+'/images'))
    y = input('Select one of above file to view images in it\n')
    path='office31/'+x+'/images/'+y
    """ for filename in os.listdir(path):
        f = os.path.join(path, filename)
        filterwarnings("ignore")
        tf_img = io.read_file(f)
        tf_img = image.decode_png(tf_img, channels=3)
        plt.imshow(tf_img)
        plt.show()"""
except:
    print('Invalid argument entered')

['webcam', 'dslr', 'amazon']
Select any one out of these

dslr
['trash_can', 'projector', 'ring_binder', 'punchers', 'ruler', 'speaker', 'scissors', 'stapler', 'tape_dispenser', 'letter_tray', 'monitor', 'printer', 'paper_notebook', 'pen', 'mouse', 'laptop_computer', 'mobile_phone', 'phone', 'mug', 'desk_lamp', 'file_cabinet', 'calculator', 'desktop_computer', 'bike_helmet', 'headphones', 'desk_chair', 'keyboard', 'bottle', 'bookcase', 'back_pack', 'bike']
Select one of above file to view images in it
mug


In [ ]:
# lets view the selected path
path

'office31/dslr/images/mug'

## Implementing Grad-CAMs

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image

In [ ]:
# Defining the model

model_builder = keras.applications.xception.Xception
img_size = (299, 299)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = "block14_sepconv2_act"
# The local path to our target image

In [ ]:
# Converting Images to array
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

In [ ]:
# Making Gradient CAM Heatmaps
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
# Combining Heatmaps and Images
def display_gradcam(img_path, heatmap, cam_path, alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    im = Image.open(cam_path)
    display(im)


In [ ]:
# Make model
model = model_builder(weights="imagenet")
model.layers[-1].activation = None

# Implementing Grad-CAMs
for filename in os.listdir(path):
    img_path = path +'/'+filename
    # Prepare image
    img_array = preprocess_input(get_img_array(img_path, size=img_size))

    # Print what the two top predicted classes are
    preds = model.predict(img_array)
    print("Predicted:", decode_predictions(preds, top=2)[0])
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=260)
    display_gradcam(img_path, heatmap , filename)
    os.remove(filename)